In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Tomato-Dataset

/content/drive/MyDrive/Tomato-Dataset


In [ ]:
!pip3 install jedi>=0.10
!pip3 install openmim
!mim install mmcv-full
!pip install light-the-torch
!ltt install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 13.3 MB/s eta 0:00:00
  Attempting uninstall: pygments
    Found existing installation: Pygments 2.6.1
    Uninstalling Pygments-2.6.1:
      Successfully uninstalled Pygments-2.6.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu116/torch1.13.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.2/190.2 KB 5.6 MB/s eta 0:00:00


In [ ]:
#!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -e .

/content/drive/MyDrive/Tomato-Dataset/mmdetection
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/Tomato-Dataset/mmdetection
  Preparing metadata (setup.py) ... done
  Running setup.py develop for mmdet


In [ ]:
import mmdet
import mmcv
import os

from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.apis import set_random_seed
from mmcv import Config

print(mmdet.__version__)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.8/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


2.28.1


In [ ]:
cfg = Config.fromfile('/content/drive/MyDrive/Tomato-Dataset/mmdetection/configs/mask_rcnn/tomato_custom_mask_rcnn.py')
dataset_type = 'CocoDataset'

cfg.data.val.ann_file = '/content/drive/MyDrive/Tomato-Dataset/Big_tomatoes_dataset-2/annotations/valid.json'
cfg.data.train.ann_file = '/content/drive/MyDrive/Tomato-Dataset/Big_tomatoes_dataset-2/annotations/train.json'
cfg.data.test.ann_file = '/content/drive/MyDrive/Tomato-Dataset/Big_tomatoes_dataset-2/annotations/test.json'

cfg.data.test.img_prefix = '/content/drive/MyDrive/Tomato-Dataset/Big_tomatoes_dataset-2/test/'
cfg.data.train.img_prefix = '/content/drive/MyDrive/Tomato-Dataset/Big_tomatoes_dataset-2/train/'
cfg.data.val.img_prefix = '/content/drive/MyDrive/Tomato-Dataset/Big_tomatoes_dataset-2/valid/'

cfg.data.test.classes = ('b_fully_ripened', 'b_green', 'b_half_ripened',)
cfg.data.train.classes = ('b_fully_ripened', 'b_green', 'b_half_ripened',)
cfg.data.val.classes = ('b_fully_ripened', 'b_green', 'b_half_ripened',)
\

cfg.load_from = '/content/drive/MyDrive/Tomato-Dataset/mmdetection/mask_rcnn_r101_caffe_fpn_1x_coco_20200601_095758-805e06c1.pth'
cfg.work_dir = '/content/drive/MyDrive/Tomato-Dataset/mmdetection/results/Results'

cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 10
#cfg.device='cpu'
cfg.device='cuda'

# Set seed thus the results are more reproducible
cfg.seed = 1221410
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# We can also use tensorboard to log the training process
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='TensorboardLoggerHook')]


print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='MaskRCNN',
    backbone=dict(
        type='ResNet',
        depth=101,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=False),
        norm_eval=True,
        style='caffe',
        init_cfg=dict(
            type='Pretrained',
            checkpoint='open-mmlab://detectron2/resnet101_caffe')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='Cro

In [ ]:
# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model)

# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(os.path.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


2023-02-21 17:28:11,541 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.


loading annotations into memory...


2023-02-21 17:28:11,807 - mmdet - INFO - load checkpoint from local path: /content/drive/MyDrive/Tomato-Dataset/mmdetection/results/Results1/latest.pth


Done (t=0.25s)
creating index...
index created!


...

DONE (t=0.24s).


/content/drive/MyDrive/Tomato-Dataset/mmdetection/mmdet/datasets/coco.py:470: UserWarning: The key "bbox" is deleted for more accurate mask AP of small/medium/large instances since v2.12.0. This does not change the overall mAP calculation.
  warnings.warn(


Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *segm*


2023-02-21 20:26:56,888 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.630
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.834
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.697
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.392
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.589
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.783
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.772
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.772
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.772
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.609
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.723
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=0.27s).
Accumulating evaluation results...
DONE (t=0.07s).


In [ ]:
# PREDICT
%cd /content/drive/MyDrive/Tomato-Dataset/mmdetection

!python tools/test.py\
configs/mask_rcnn/tomato_custom_mask_rcnn.py\
/content/drive/MyDrive/Tomato-Dataset/mmdetection/results/Results3/epoch_1.pth\
--show-score-thr 0.5\
--show-dir /content/drive/MyDrive/Tomato-Dataset/mmdetection/results/Results3/eval\
--format-only\
--options 'jsonfile_prefix=/content/drive/MyDrive/Tomato-Dataset/mmdetection/results/Results1/test-dev_results'\

/content/drive/MyDrive/Tomato-Dataset/mmdetection
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.8/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
tools/test.py:116: UserWarning: --options is deprecated in favor of --eval-options
  warnings.warn('--options is deprecated in favor of --eval-options')
/content/drive/MyDrive/Tomato-Dataset/mmdetection/mmdet/utils/setup_env.py:38: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  warnings.war